In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
path='C:\Users\HP\Desktop\AI Landslide mitigation\iit-mandi.ipynb'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [45]:
from numpy import mean
from numpy import std
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
!pip install scitime

In [47]:
from scitime import Estimator 

In [ ]:
df = pd.read_csv(path)
df.head()

# EDA

In [ ]:
df.info()

In [51]:
temp_cols=df.columns.tolist()
new_cols=temp_cols[1:] + temp_cols[0:1]
df=df[new_cols]

In [ ]:
df.describe().round(2)

In [ ]:
for column in df.columns[0:]:
    print(column, ': ', len(df[column].unique()), ' labels')

In [ ]:
landslide_count = df['Landslide'].value_counts()
sns.set(style="darkgrid")
sns.barplot(landslide_count.index, landslide_count.values, alpha=0.9)
plt.title('Frequency Distribution of Landslides')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Landslide', fontsize=12)
plt.show()

plt.savefig('bar1.eps', format='eps')

In [ ]:
perc_count = df['Precipitation'].value_counts()
sns.set(style="darkgrid")
sns.barplot(perc_count.index, perc_count.values, alpha=0.9)
plt.title('Frequency Distribution of Precipitation')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Precipitation', fontsize=12)
plt.show()

plt.savefig('bar2.eps', format='eps')

In [ ]:
lith_count = df['Lithology'].value_counts()
sns.set(style="darkgrid")
sns.barplot(lith_count.index, lith_count.values, alpha=0.9)
plt.title('Frequency Distribution of Lithology')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Lithology', fontsize=12)
plt.show()

plt.savefig('bar3.eps', format='eps')

In [ ]:
labels = df['Plan'].astype('category').cat.categories.tolist()
counts = df['Plan'].value_counts()
sizes = [counts[var_cat] for var_cat in labels]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True) #autopct is show the % on plot
ax1.axis('equal')
plt.title('Pieplot of Plan')
plt.show()

plt.savefig('pie1.eps', format='eps')

In [ ]:
labels = df['Flow'].astype('category').cat.categories.tolist()
counts = df['Flow'].value_counts()
sizes = [counts[var_cat] for var_cat in labels]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True) #autopct is show the % on plot
ax1.axis('equal')
plt.title('Pieplot of Flow')
plt.show()

plt.savefig('pie2.eps', format='eps')

In [ ]:
labels = df['Curvature'].astype('category').cat.categories.tolist()
counts = df['Curvature'].value_counts()
sizes = [counts[var_cat] for var_cat in labels]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True) #autopct is show the % on plot
ax1.axis('equal')
plt.title('Pieplot of Curvature')
plt.show()

plt.savefig('pie3.eps', format='eps')

In [ ]:
sns.catplot(x="Landslide", y="NDWI", kind="swarm", data=df)

plt.savefig('box1.eps', format='eps')

In [ ]:
# sns.set_theme(style="ticks")

# sns.pairplot(df)

In [ ]:
plt.figure(figsize=(13, 13))
sns.heatmap(df.corr(), annot=True)

plt.show()

plt.savefig('corr1.eps', format='eps')

In [ ]:
df.mean()

In [ ]:
df.groupby(['Landslide']).mean()

In [ ]:
df.corr()

## Important Features using Mutual Information Classification

In [68]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [70]:
train_ratio = 0.8
val_ratio = 0.2
train_df, val_df = train_test_split(df, train_size = train_ratio, random_state=42)

In [71]:
features = list(df.columns.values.tolist())
df_feat = df.iloc[:, :-1]
features_wo = list(df_feat.columns.values.tolist())

In [ ]:
df_feat.info()

In [ ]:
from sklearn.feature_selection import mutual_info_classif

MI = (mutual_info_classif(df[features_wo], df["Landslide"], n_neighbors=20,
                          random_state=42))
 
plt.figure(figsize=(5.4, 4))
plt.barh(y=features_wo, width=MI, color="#990303")
plt.title("Mutual Information w.r.t. Landslide");
plt.xlabel("Landslide")
plt.gca().xaxis.grid(True, linestyle=':');
plt.tight_layout()

plt.savefig('mi.eps', format='eps')

In [76]:
best3 = ['Precipitation', 'Earthquake', 'Aspect']
best5 = ['Lithology', 'Flow']
best5.extend(best3)
best8 = ['Profile', 'NDWI', 'NDVI']
best8.extend(best5)
all_feat = features_wo

feat  = [best3, best5, best8, all_feat]

In [ ]:
for i in feat:
    print(df[i].head(2))

# Feature Engineering

In [ ]:
df.head(1)

## Frequency Encoding

In [80]:
Aspect_freq = df['Aspect'].value_counts(normalize=True)
df['Aspect_freq'] = df['Aspect'].apply(lambda x : Aspect_freq[x])

Curvature_freq = df['Curvature'].value_counts(normalize=True)
df['Curvature_freq'] = df['Curvature'].apply(lambda x : Curvature_freq[x])

Earthquake_freq = df['Earthquake'].value_counts(normalize=True)
df['Earthquake_freq'] = df['Earthquake'].apply(lambda x : Earthquake_freq[x])

Elevation_freq = df['Elevation'].value_counts(normalize=True)
df['Elevation_freq'] = df['Elevation'].apply(lambda x : Elevation_freq[x])

Flow_freq = df['Flow'].value_counts(normalize=True)
df['Flow_freq'] = df['Flow'].apply(lambda x : Flow_freq[x])

Lithology_freq = df['Lithology'].value_counts(normalize=True)
df['Lithology_freq'] = df['Lithology'].apply(lambda x : Lithology_freq[x])

NDVI_freq = df['NDVI'].value_counts(normalize=True)
df['NDVI_freq'] = df['NDVI'].apply(lambda x : NDVI_freq[x])

NDWI_freq = df['NDWI'].value_counts(normalize=True)
df['NDWI_freq'] = df['NDWI'].apply(lambda x : NDWI_freq[x])

Plan_freq = df['Plan'].value_counts(normalize=True)
df['Plan_freq'] = df['Plan'].apply(lambda x : Plan_freq[x])

Precipitation_freq = df['Precipitation'].value_counts(normalize=True)
df['Precipitation_freq'] = df['Precipitation'].apply(lambda x : Precipitation_freq[x])

Profile_freq = df['Profile'].value_counts(normalize=True)
df['Profile_freq'] = df['Profile'].apply(lambda x : Profile_freq[x])

Slope_freq = df['Slope'].value_counts(normalize=True)
df['Slope_freq'] = df['Slope'].apply(lambda x : Slope_freq[x])

In [81]:
# Complete dataframe with frequency variables and target variable
df_comp = df.iloc[:, :13]

# Dataframe with frequency variables and target variable
df_freq = df.iloc[:, 12:]

# Dataframe with frequency variables
df_onlyfreq = df.iloc[:, 13:]

In [ ]:
plt.figure(figsize=(13, 13))
sns.heatmap(df_freq.corr(), annot=True)

plt.show()

plt.savefig('corr2.eps', format='eps')

## Feature Selection

In [84]:
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn import ensemble

In [ ]:
!pip install pca

In [88]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from pca import pca

X_pca = df.loc[:, df.columns != 'Landslide']

# scaler = StandardScaler()
# scaler.fit(X_pca)
# X_pca = scaler.transform(X_pca)

model = pca()
out = model.fit_transform(X_pca)

print(out['topfeat'])

In [ ]:
model.plot()

plt.savefig('pca1.eps', format='eps')

In [ ]:
ax = model.biplot(n_feat=10, legend=False)

plt.savefig('pca2.eps', format='eps')

In [ ]:
ax = model.biplot3d(n_feat=10, legend=False)

plt.savefig('pca3.eps', format='eps')

# Baseline Model

In [94]:
y = df.Landslide
# y = df_freq.Landslide
# df= df.loc[:, df.columns != 'Landslide']
df1 = df.loc[:, df.columns != 'Landslide']
# df1 = df.loc[:, ['NDWI', 'Curvature', 'Lithology', 'Precipitation', 'Aspect', 'Flow', 
#                 'Elevation', 'Profile', 'Earthquake']]
# df1 = df_freq.loc[:, ['Lithology_freq', 'Flow_freq', 'Precipitation_freq', 'Earthquake_freq', 'Curvature_freq',
#                 'Profile_freq', 'NDVI_freq', 'Aspect_freq']]
X_train, X_test, y_train, y_test = train_test_split(df1, y, test_size=0.2)

In [ ]:
baseline_gbm = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, 
                                          min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)
baseline_gbm.fit(X_train,y_train)
predictors_gbm=list(X_train)
# feat_imp_gbm = pd.Series(baseline_gbm.feature_importances_, predictors_gbm).sort_values(ascending=False)
# feat_imp_gbm.plot(kind='bar', title='Importance of Features Using GBM')
# plt.ylabel('Feature Importance Score')
print('Accuracy of the GBM on test set: {:.3f}'.format(baseline_gbm.score(X_test, y_test)))
pred_gbm=baseline_gbm.predict(X_test)
print(classification_report(y_test, pred_gbm))

In [ ]:
baseline_lgbm = LGBMClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, 
                          #min_samples_split=2, 
                          min_samples_leaf=1, subsample=1,
                          #max_features='sqrt', 
                          random_state=10)
baseline_lgbm.fit(X_train,y_train)
predictors_lgbm=list(X_train)
# feat_imp_lgbm = pd.Series(baseline_lgbm.feature_importances_, predictors_lgbm).sort_values(ascending=False)
# feat_imp_lgbm.plot(kind='bar', title='Importance of Features Using LGBM')
# plt.ylabel('Feature Importance Score')
print('Accuracy of the LGBM on test set: {:.3f}'.format(baseline_lgbm.score(X_test, y_test)))
pred_lgbm=baseline_lgbm.predict(X_test)
print(classification_report(y_test, pred_lgbm))

In [97]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



In [98]:
# Create a Random Forest Classifier
random_forest = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=10)



In [ ]:
# Fit the model to the training data
random_forest.fit(X_train, y_train)

# Make predictions on the test set
pred_rf = random_forest.predict(X_test)

# Calculate and print the accuracy
accuracy_rf = random_forest.score(X_test, y_test)
print('Accuracy of the Random Forest on test set: {:.3f}'.format(accuracy_rf))

# Generate and print the classification report
print(classification_report(y_test, pred_rf))


In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report

# Create an XGBoost Classifier
xgb_classifier = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=3,
    min_child_weight=1,
    gamma=0,
    subsample=1,
    colsample_bytree=1,
    objective='binary:logistic',  # For binary classification
    random_state=10
)

# Fit the model to the training data
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
pred_xgb = xgb_classifier.predict(X_test)

# Calculate and print the accuracy
accuracy_xgb = xgb_classifier.score(X_test, y_test)
print('Accuracy of XGBoost on test set: {:.3f}'.format(accuracy_xgb))

# Generate and print the classification report
print(classification_report(y_test, pred_xgb))


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report

# Define individual models
gbm_model = GradientBoostingClassifier(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=1,
    max_features='sqrt',
    random_state=10
)

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=3,
    random_state=10
)

# Create an ensemble of models
ensemble_model = VotingClassifier(
    estimators=[
        ('gbm', gbm_model),
        ('rf', rf_model),
        # Add more models here if desired
    ],
    voting='soft'  # 'soft' for weighted voting based on probabilities
)

# Fit the ensemble model on the training data
ensemble_model.fit(X_train, y_train)

# Make predictions on the test set
pred_ensemble = ensemble_model.predict(X_test)

# Calculate and print the accuracy
accuracy_ensemble = ensemble_model.score(X_test, y_test)
print('Accuracy of the Ensemble Model on test set: {:.3f}'.format(accuracy_ensemble))

# Generate and print the classification report
print(classification_report(y_test, pred_ensemble))


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Create an SVM classifier
svm_classifier = SVC(kernel='rbf', C=1.0, random_state=10)

# Fit the SVM model to the training data
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
pred_svm = svm_classifier.predict(X_test)

# Calculate and print the accuracy
accuracy_svm = svm_classifier.score(X_test, y_test)
print('Accuracy of the SVM on test set: {:.3f}'.format(accuracy_svm))

# Generate and print the classification report
print(classification_report(y_test, pred_svm))


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report

# Define the GBM model (you can use your existing model)
gbm_model = GradientBoostingClassifier(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=1,
    max_features='sqrt',
    random_state=10
)

# Define the SVM model
svm_model = SVC(probability=True, random_state=10)  # Use probability=True to enable soft voting

# Create an ensemble of models
ensemble_model = VotingClassifier(
    estimators=[
        ('gbm', gbm_model),
        ('svm', svm_model)
        # Add more models here if desired
    ],
    voting='soft'  # 'soft' for weighted voting based on probabilities
)

# Fit the ensemble model on the training data
ensemble_model.fit(X_train, y_train)

# Make predictions on the test set
pred_ensemble = ensemble_model.predict(X_test)

# Calculate and print the accuracy
accuracy_ensemble = ensemble_model.score(X_test, y_test)
print('Accuracy of the Ensemble Model on test set: {:.3f}'.format(accuracy_ensemble))

# Generate and print the classification report
print(classification_report(y_test, pred_ensemble))


# Tuning of Models

## n_estimators and Learning Rate

In [111]:
p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[50,100,250,500,750,1000,1250,1500,1750]}

tuning = GridSearchCV(estimator =GradientBoostingClassifier(max_depth=4, min_samples_split=2, min_samples_leaf=1, 
                                                            subsample=1,max_features='sqrt', random_state=10), 
                                                            param_grid = p_test3, scoring='accuracy',n_jobs=4,
                                                            iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
learning_rate_gbm = tuning.best_params_.get("learning_rate")
n_estimators_gbm = tuning.best_params_.get("n_estimators")

In [ ]:
p_test3a = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[50,100,250,500,750,1000,1250,1500,1750]}

tuning = GridSearchCV(estimator =LGBMClassifier(max_depth=4, min_samples_leaf=1, 
                                                subsample=1, random_state=10), 
                                                param_grid = p_test3a, scoring='accuracy',n_jobs=4,
                                                iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
learning_rate_lgbm = tuning.best_params_.get("learning_rate")
n_estimators_lgbm = tuning.best_params_.get("n_estimators")

## max_depth

In [112]:
p_test2 = {'max_depth':[2,3,4,5,6,7] }
tuning = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=learning_rate_gbm,n_estimators=n_estimators_gbm,
                                                            min_samples_split=2, 
                                                            min_samples_leaf=1, subsample=1,max_features='sqrt',
                                                            random_state=10), 
                                                            param_grid = p_test2, scoring='accuracy',n_jobs=4,
                                                            iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
max_depth_gbm = tuning.best_params_.get("max_depth")

In [113]:
p_test2a = {'max_depth':[2,3,4,5,6,7] }
tuning = GridSearchCV(estimator =LGBMClassifier(learning_rate=learning_rate_lgbm,n_estimators=n_estimators_lgbm, 
                                                min_samples_leaf=1, subsample=1, random_state=10), 
                                                param_grid = p_test2, scoring='accuracy',n_jobs=4,
                                                iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
max_depth_lgbm = tuning.best_params_.get("max_depth")

## First Evaluation of New Model on Test Set

In [ ]:
model1 = GradientBoostingClassifier(learning_rate=learning_rate_gbm, n_estimators=n_estimators_gbm,max_depth=max_depth_gbm, 
                                    min_samples_split=2, 
                                    min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)
model1.fit(X_train,y_train)
predictors=list(X_train)
# feat_imp = pd.Series(model1.feature_importances_, predictors).sort_values(ascending=False)
# feat_imp.plot(kind='bar', title='Importance of Features')
# plt.ylabel('Feature Importance Score')
print('Accuracy of the GBM on test set: {:.3f}'.format(model1.score(X_test, y_test)))
pred=model1.predict(X_test)
print(classification_report(y_test, pred))

In [ ]:
model1_lgbm = LGBMClassifier(learning_rate=learning_rate_lgbm, n_estimators=n_estimators_lgbm,max_depth=max_depth_lgbm,  
                             min_samples_leaf=1, subsample=1, random_state=10)
model1_lgbm.fit(X_train,y_train)
predictors=list(X_train)
# feat_imp = pd.Series(model1_lgbm.feature_importances_, predictors).sort_values(ascending=False)
# feat_imp.plot(kind='bar', title='Importance of Features')
# plt.ylabel('Feature Importance Score')
print('Accuracy of the LGBM on test set: {:.3f}'.format(model1_lgbm.score(X_test, y_test)))
pred=model1_lgbm.predict(X_test)
print(classification_report(y_test, pred))

## Min Sample Split and Min Samples Leaf

In [117]:
p_test4 = {'min_samples_split':[2,4,6,8,10,20,40,60,100], 'min_samples_leaf':[1,3,5,7,9]}

tuning = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=learning_rate_gbm, n_estimators=n_estimators_gbm,
                                                            max_depth=max_depth_gbm, 
                                                            subsample=1,max_features='sqrt', random_state=10), 
                                                            param_grid = p_test4, scoring='accuracy',n_jobs=4,
                                                            iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
min_samples_leaf_gbm = tuning.best_params_.get("min_samples_leaf")

In [ ]:
p_test4a = {'min_samples_leaf':[1,3,5,7,9]}

tuning = GridSearchCV(estimator =LGBMClassifier(learning_rate=learning_rate_lgbm, n_estimators=n_estimators_lgbm,
                                                max_depth=max_depth_lgbm, 
                                                subsample=1, random_state=10), 
                                                param_grid = p_test4a, scoring='accuracy',n_jobs=4,
                                                iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
min_samples_leaf_lgbm = tuning.best_params_.get("min_samples_leaf")

## Max Features

In [119]:
p_test5 = {'max_features':[2,3,4,5,6,7]}
tuning = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=learning_rate_gbm, n_estimators=n_estimators_gbm,
                                                            max_depth=max_depth_gbm, 
                                                            min_samples_split=20, min_samples_leaf=min_samples_leaf_gbm, 
                                                            subsample=1, 
                                                            random_state=10), param_grid = p_test5, scoring='accuracy',
                                                            n_jobs=4,iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
max_features_gbm = tuning.best_params_.get("max_features")

## Subsample

In [125]:
p_test6= {'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1]}

tuning = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=learning_rate_gbm, n_estimators=n_estimators_gbm,
                                                            max_depth=max_depth_gbm, 
                                                            min_samples_split=20, min_samples_leaf=min_samples_leaf_gbm,
                                                            max_features=max_features_gbm , 
                                                            random_state=10), param_grid = p_test6, scoring='accuracy',
                                                            n_jobs=4,iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
subsample_gbm = tuning.best_params_.get("subsample")

In [126]:
p_test6a= {'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1]}

tuning = GridSearchCV(estimator =LGBMClassifier(learning_rate=learning_rate_lgbm, n_estimators=n_estimators_lgbm,
                                                max_depth=max_depth_lgbm, 
                                                min_samples_leaf=min_samples_leaf_lgbm,
                                                random_state=10), param_grid = p_test6, scoring='accuracy',
                                                n_jobs=4,iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
subsample_lgbm = tuning.best_params_.get("subsample")

## random_state

In [127]:
p_test7= {'random_state':list(range(0,101,2))}

tuning = GridSearchCV(estimator =GradientBoostingClassifier(learning_rate=learning_rate_gbm, n_estimators=n_estimators_gbm,
                                                            max_depth=max_depth_gbm, 
                                                            min_samples_split=20, min_samples_leaf=min_samples_leaf_gbm,
                                                            max_features=max_features_gbm , 
                                                            subsample=subsample_gbm), param_grid = p_test7, 
                                                            scoring='accuracy',
                                                            n_jobs=4,iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
random_state_gbm = tuning.best_params_.get("random_state")

In [ ]:
p_test7a= {'random_state':list(range(0,101,2))}

tuning = GridSearchCV(estimator =LGBMClassifier(learning_rate=learning_rate_lgbm, n_estimators=n_estimators_lgbm,
                                                max_depth=max_depth_lgbm, min_samples_leaf=min_samples_leaf_lgbm, 
                                                subsample=subsample_lgbm), param_grid = p_test7a, scoring='accuracy',
                                                n_jobs=4,iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_
random_state_lgbm = tuning.best_params_.get("random_state")

## Optimal Parameters

In [129]:
a1 = []
a2 = []
a3 = []
a4 = []
a5 = []
a6 = []
a7 = []
a8 = []

In [132]:
a1.append('GBM')
a2.append(learning_rate_gbm)
a3.append(n_estimators_gbm)
a4.append(max_depth_gbm)
a5.append(min_samples_leaf_gbm)
a6.append(max_features_gbm)
a7.append(subsample_gbm)
a8.append(random_state_gbm)

opt_par = pd.DataFrame({'Name': a1, 'learning_rate': a2, 'n_estimators': a3,
                         'max_depth': a4, 'min_samples_leaf': a5, 'max_features': a6,
                         'subsample_gbm': a7, 'random_state': a8})

In [134]:
a1.append('LGBM')
a2.append(learning_rate_lgbm)
a3.append(n_estimators_lgbm)
a4.append(max_depth_lgbm)
a5.append(min_samples_leaf_lgbm)
a6.append('--')
a7.append(subsample_lgbm)
a8.append(random_state_lgbm)

opt_par = pd.DataFrame({'Name': a1, 'learning_rate': a2, 'n_estimators': a3,
                         'max_depth': a4, 'min_samples_leaf': a5, 'max_features': a6,
                         'subsample_gbm': a7, 'random_state': a8})

In [ ]:
opt_par

# Evaluation of Final Model on Test Set

In [ ]:
new=GradientBoostingClassifier(learning_rate=learning_rate_gbm, n_estimators=n_estimators_gbm,
                               max_depth=max_depth_gbm, min_samples_split=20, min_samples_leaf=min_samples_leaf_gbm,
                               max_features=max_features_gbm, subsample=subsample_gbm, random_state=random_state_gbm)
new.fit(X_train,y_train)
predictors=list(X_train)
# feat_imp = pd.Series(new.feature_importances_, predictors).sort_values(ascending=False)
# feat_imp.plot(kind='bar', title='Importance of Features')
# plt.ylabel('Feature Importance Score')
print('Accuracy of the GBM on test set: {:.3f}'.format(new.score(X_test, y_test)))
pred=new.predict(X_test)
print(classification_report(y_test, pred))

In [ ]:
new_lgbm=LGBMClassifier(learning_rate=learning_rate_lgbm, n_estimators=n_estimators_lgbm,max_depth=max_depth_lgbm,
                        min_samples_leaf=min_samples_leaf_lgbm, subsample=subsample_lgbm, random_state=random_state_lgbm)
new_lgbm.fit(X_train,y_train)
predictors=list(X_train)
# feat_imp = pd.Series(new_lgbm.feature_importances_, predictors).sort_values(ascending=False)
# feat_imp.plot(kind='bar', title='Importance of Features')
# plt.ylabel('Feature Importance Score')
print('Accuracy of the LGBM on test set: {:.3f}'.format(new_lgbm.score(X_test, y_test)))
pred=new_lgbm.predict(X_test)
print(classification_report(y_test, pred))

# Comparison of ROC AUC

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve 


#run models
baseline = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, 
                                      min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)
baseline.fit(X_train,y_train)

model1 = GradientBoostingClassifier(learning_rate=learning_rate_gbm, n_estimators=n_estimators_gbm,max_depth=max_depth_gbm, 
                                    min_samples_split=2, 
                                    min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)
model1.fit(X_train,y_train)

new=GradientBoostingClassifier(learning_rate=learning_rate_gbm, n_estimators=n_estimators_gbm,
                               max_depth=max_depth_gbm, min_samples_split=20, min_samples_leaf=min_samples_leaf_gbm,
                               max_features=max_features_gbm, subsample=subsample_gbm, random_state=random_state_gbm)
new.fit(X_train,y_train)

#calculate values for ROC AUC plot
#baseline model
baseline_roc_auc = roc_auc_score(y_test, baseline.predict(X_test))
fprB, tprB, thresholdsB = roc_curve(y_test, baseline.predict_proba(X_test)[:,1])
#model 1
model1_roc_auc = roc_auc_score(y_test, model1.predict(X_test))
fpr1, tpr1, thresholds1 = roc_curve(y_test, model1.predict_proba(X_test)[:,1])
#new tuned model 
new_roc_auc = roc_auc_score(y_test, new.predict(X_test))
fprnew, tprnew, thresholds_new = roc_curve(y_test, new.predict_proba(X_test)[:,1])

plt.figure()
plt.plot(fprB, tprB, label='GBM Baseline (area = %0.2f)' % baseline_roc_auc)
plt.plot(fpr1, tpr1, label='GBM Model 1 (area = %0.2f)' % model1_roc_auc)
plt.plot(fprnew, tprnew, label='GBM Final Model (area = %0.2f)' % new_roc_auc)

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic of GBM')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

plt.savefig('roc1.eps', format='eps')

In [ ]:
print('Accuracy of the GBM on test set for Baseline Model: {:.3f}'.format(baseline.score(X_test, y_test)))
print('Accuracy of the GBM on test set for Model1: {:.3f}'.format(model1.score(X_test, y_test)))
print('Accuracy of the GBM on test set for New Model: {:.3f}'.format(new.score(X_test, y_test)))

In [ ]:
#run models
baseline_lgbm = LGBMClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, 
                               min_samples_leaf=1, subsample=1, random_state=10)
baseline_lgbm.fit(X_train,y_train)

model1_lgbm = LGBMClassifier(learning_rate=learning_rate_lgbm, n_estimators=n_estimators_lgbm,max_depth=max_depth_lgbm,  
                             min_samples_leaf=1, subsample=1, random_state=10)
model1_lgbm.fit(X_train,y_train)

new_lgbm=LGBMClassifier(learning_rate=learning_rate_lgbm, n_estimators=n_estimators_lgbm,max_depth=max_depth_lgbm,
                        min_samples_leaf=min_samples_leaf_lgbm, subsample=subsample_lgbm, random_state=random_state_lgbm)
new_lgbm.fit(X_train,y_train)

#calculate values for ROC AUC plot
#baseline model
baseline_lgbm_roc_auc = roc_auc_score(y_test, baseline_lgbm.predict(X_test))
fprB, tprB, thresholdsB = roc_curve(y_test, baseline_lgbm.predict_proba(X_test)[:,1])
#model 1
model1_lgbm_roc_auc = roc_auc_score(y_test, model1_lgbm.predict(X_test))
fpr1, tpr1, thresholds1 = roc_curve(y_test, model1_lgbm.predict_proba(X_test)[:,1])
#new tuned model 
new_lgbm_roc_auc = roc_auc_score(y_test, new_lgbm.predict(X_test))
fprnew, tprnew, thresholds_new = roc_curve(y_test, new_lgbm.predict_proba(X_test)[:,1])

plt.figure()
plt.plot(fprB, tprB, label='LGBM Baseline (area = %0.2f)' % baseline_roc_auc)
plt.plot(fpr1, tpr1, label='LGBM Model 1 (area = %0.2f)' % model1_roc_auc)
plt.plot(fprnew, tprnew, label='LGBM Final Model (area = %0.2f)' % new_roc_auc)

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic of LGBM')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

plt.savefig('roc2.eps', format='eps')

In [ ]:
print('Accuracy of the LGBM on test set for Baseline Model: {:.3f}'.format(baseline_lgbm.score(X_test, y_test)))
print('Accuracy of the LGBM on test set for Model1: {:.3f}'.format(model1_lgbm.score(X_test, y_test)))
print('Accuracy of the LGBM on test set for New Model: {:.3f}'.format(new_lgbm.score(X_test, y_test)))